# Silver to Gold

## Importing the modules, files, and setting oAuth

In [ ]:
## Importing modules
import os
import datetime
import re
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from datetime import datetime
from datetime import timezone
import pandas as pd
from sparknlp.base import *
from sparknlp.annotator import *


# to_date() function for translating dates
spark = SparkSession.builder.appName('PySpark to_date()').getOrCreate()

# Legacy mode to be able to use date_format()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [ ]:
## Getting oAuth for Azure Container
service_credential = dbutils.secrets.get(scope="databricks",key="databricks-test")
# spark.conf here

In [ ]:
## Reading necessary parquet files from silver layer 
main_df  = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/main_df")
push_event_df  = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/push_event_df")
push_event_df_commits_df  = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/push_event_df_commits_df")
create_event_df  = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/create_event_df")
commit_comment_event_df  = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/commit_comment_event_df")
issue_comment_event_df = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/issue_comment_event_df")
pull_request_review_comment_event_df = spark.read.parquet(f"abfs://silver-layer@ade20220919.dfs.core.windows.net/miniro/pull_request_review_comment_event_df")

## Type of GitHub event per hour

In [ ]:
# converting the UTC time stamp to a more readable format 
# and saving it to its own table
time_event_df = main_df\
    .select("id", "type", "created_at")\
    .withColumn("hour", 
            F.date_format(F.to_timestamp(F.col("created_at")),
            "MM/dd/yyyy HH:00:00"))

# grouping the hour and type columns and ordering by hour
# and then getting the count of each event type for each hour 
event_per_hour_df = time_event_df \
    .select("id", "type", "created_at", "hour") \
    .groupBy("hour") \
    .pivot("type") \
    .count() \
    .drop('type') \
    .orderBy("hour")

event_per_hour_df.display()

hour,CommitCommentEvent,CreateEvent,DeleteEvent,ForkEvent,GollumEvent,IssueCommentEvent,IssuesEvent,MemberEvent,PublicEvent,PullRequestEvent,PullRequestReviewCommentEvent,PushEvent,ReleaseEvent,WatchEvent
01/01/2017 00:00:00,38,2172,572,347,92,864,488,21,13,708,108,7898,71,987
01/01/2017 01:00:00,23,2579,344,446,40,794,413,20,38,648,93,8054,72,1081
01/01/2017 02:00:00,30,5386,626,430,65,718,403,21,13,612,112,8300,36,1310
01/01/2017 03:00:00,46,2137,352,444,114,679,363,22,7,588,94,7748,58,1322
01/01/2017 04:00:00,28,1864,364,457,99,586,342,29,6,525,72,7679,38,1265
01/01/2017 05:00:00,26,1891,431,417,147,505,316,21,9,462,101,7642,37,1329
01/01/2017 06:00:00,15,1982,440,452,112,439,353,30,9,405,60,6823,35,1220
01/01/2017 07:00:00,20,2046,247,462,68,613,345,34,10,484,139,6793,35,1421
01/01/2017 08:00:00,23,2262,421,499,55,625,457,26,8,518,113,7478,45,1399
01/01/2017 09:00:00,30,2124,460,501,56,731,514,33,8,559,169,8006,48,1407


In [ ]:
# writing to a file in gold 
event_per_hour_df.write.mode("overwrite") \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/event_per_hour_df')

## PushEvents on main/gh-pages/other branches

In [ ]:
# PushEvent data aggregated by ref type – whether the commit is on the main branch 
push_event_ref_df = push_event_df \
    .withColumn("day", F.date_format(F.to_timestamp(F.col("created_at")),
                "MM/dd/yyyy")) \
    .withColumn("ma", (F.split(push_event_df.ref, "/")[2] == "master")) \
    .withColumn("gp", (F.split(push_event_df.ref, "/")[2] == "gh-pages")) \
    .withColumn("ot", ((F.split(push_event_df.ref, "/")[2] != "master") &
                        (F.split(push_event_df.ref, "/")[2] != "gh-pages"))) \
    .groupBy(F.col("day")) \
    .agg( \
        (F.sum(F.col("ma").cast("long"))).alias("main"), \
        (F.sum(F.col("gp").cast("long"))).alias("gh-pages"), \
        (F.sum(F.col("ot").cast("long"))).alias("other")) \
    .orderBy(F.col("day").asc())

push_event_ref_df.display()

day,main,gh-pages,other
01/01/2017,170369,7720,55462
01/02/2017,270904,11567,80937
01/03/2017,345536,14213,121397
01/04/2017,365793,14339,127106
01/05/2017,333742,13429,124723
01/06/2017,340161,13343,134186
01/07/2017,263556,10304,98935
01/08/2017,278878,11340,90547
01/09/2017,373435,15682,136711
01/10/2017,382993,14716,132628


In [ ]:
# writing to a file in gold
push_event_ref_df.write.mode("overwrite") \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/push_event_ref_df')

## Number of commits per PushEvent

In [ ]:
# creating a number of commits data frame with
# formatted time 
# use distinct_size instead since commits array counts for the 20 most recent commits 

num_commits_df = push_event_df.join(main_df.drop("created_at"), "id")

num_commits_df = num_commits_df \
    .withColumn("branch",F.split(push_event_df.ref, "/" [2])\
    .select(\
        F.col("id").alias("fact_id"), "push_id",
        F.date_format(F.to_timestamp(F.col("created_at")), "MM/dd/yyyy HH:mm:ss").alias("time"),
        F.col("actor_display_login").alias("user"), 
        F.col("org_login").alias("organization"), 
        F.col("repo_name").alias("repository"),
        F.col("branch"),
        F.col("distinct_size").alias('number_commits'))
        

num_commits_df.display()

fact_id,push_id,time,user,organization,repository,branch,number_commits
5088648464,1479378584,01/01/2017 00:00:04,bhollis,DestinyItemManager,DestinyItemManager/DIM,dev,11
5088648630,1479378656,01/01/2017 00:00:09,EdgarVaguencia,null,EdgarVaguencia/activeGitHub,master,1
5088649114,1479378799,01/01/2017 00:00:25,jlippold,null,jlippold/Home.Automation,master,1
5088649215,1479378827,01/01/2017 00:00:30,shgtkshruch,null,shgtkshruch/shgtkshruch,master,1
5088649225,1479378831,01/01/2017 00:00:30,mrocklin,null,mrocklin/sortedcollections-feedstock,rerender,1
5088649250,1479378839,01/01/2017 00:00:31,ChALkeR,null,ChALkeR/LTS,eol-0.12,1
5088649594,1479378945,01/01/2017 00:00:57,syclik,stan-dev,stan-dev/stan-dev.github.io,master,1
5088649698,1479378979,01/01/2017 00:01:04,kasvestas,itminedu,itminedu/osguide,master,5
5088649881,1479379041,01/01/2017 00:01:17,pascal-fb-martin,null,pascal-fb-martin/motionCenter,master,4
5088649985,1479379073,01/01/2017 00:01:23,raspberrypisig,null,raspberrypisig/raspberrypisig.github.io,master,1


In [ ]:
# save num_commits_df to parquet 
num_commits_df.write.mode('overwrite') \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/num_commits_df')

## User user activity by week

In [ ]:
# displaying our main dataframe 
main_df.select("actor_id", "actor_display_login", "type", "created_at") \
    .display()

actor_id,actor_display_login,type,created_at
5214914,denis-bogdanas,IssuesEvent,2017-01-14T02:46:14Z
8701921,latobarita,PushEvent,2017-01-14T06:20:03Z
16975522,jonpants,ForkEvent,2017-01-14T02:45:59Z
24866853,thinkbotbot,PushEvent,2017-01-14T08:20:35Z
24866853,thinkbotbot,CreateEvent,2017-01-14T02:39:38Z
24866853,thinkbotbot,CreateEvent,2017-01-14T06:30:58Z
18504030,melaniechio,PushEvent,2017-01-14T02:38:25Z
24866853,thinkbotbot,CreateEvent,2017-01-14T06:26:22Z
11483661,Blood-PawWerewolf,WatchEvent,2017-01-14T02:15:15Z
24866853,thinkbotbot,PushEvent,2017-01-14T06:26:14Z


In [ ]:
 # formatting the UTC timestamp 
 # Grouping by the week of the year
 # filling nulls with zero 
user_type_week = main_df \
    .select("actor_id", "actor_display_login", 
            "type", "created_at") \
    .withColumn("created_at", F.to_timestamp("created_at")) \
    .withColumn("date", F.to_date("created_at")) \
    .withColumn("week_of_year", F.date_format(F.to_date("date", "dd/MMM/yyyy"), "w")) \
    .groupBy('week_of_year', 'actor_display_login') \
    .pivot('type') \
    .count() \
    .fillna(0) \
    .orderBy('week_of_year',  'actor_display_login') \
    .withColumnRenamed('actor_display_login', 'user')

week_of_year,user,CommitCommentEvent,CreateEvent,DeleteEvent,ForkEvent,GollumEvent,IssueCommentEvent,IssuesEvent,MemberEvent,PublicEvent,PullRequestEvent,PullRequestReviewCommentEvent,PushEvent,ReleaseEvent,WatchEvent
1,0--key,1,1,0,0,0,0,0,0,0,2,0,14,0,0
1,0-T-0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
1,0-afflatus,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0-elip,1,0,0,0,0,0,0,0,0,0,0,8,0,0
1,0-kaladin,0,0,0,0,0,1,1,0,0,0,0,1,0,0
1,0-php,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,0-wiz-0,0,0,0,0,0,1,1,0,0,0,0,19,0,0
1,00-00-00,0,2,0,0,0,0,0,0,0,0,0,0,0,0
1,00000111,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,00000maik0010,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# example of filtering by user 
user_type_week.where(F.col('user') == 'torodev').display()

week_of_year,user,CommitCommentEvent,CreateEvent,DeleteEvent,ForkEvent,GollumEvent,IssueCommentEvent,IssuesEvent,MemberEvent,PublicEvent,PullRequestEvent,PullRequestReviewCommentEvent,PushEvent,ReleaseEvent,WatchEvent
1,torodev,0,0,1,0,0,1,1,0,0,0,0,1,0,0
2,torodev,44,153,83,37,0,94,105,47,0,119,53,247,38,0
3,torodev,1,2,1,1,0,1,1,1,0,0,1,4,1,0
4,torodev,1,2,1,1,0,1,1,1,0,0,1,4,0,0


In [ ]:
# save user_type_date to parquet
user_type_week.write.mode("overwrite") \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/user_type_week')

## Breakdown of activity by project – find a unique use case

In [ ]:
## Joining Push Event Table with Fact Table
repo_push_df = push_event_df \
    .select(F.date_format(
                F.to_timestamp("created_at"), "MM/dd/yyyy HH:00:00") \
                .alias('created_at'), 'id', 'push_id', 'distinct_size') \
    .join(
        main_df.select(
            'id', 'repo_name', "repo_id", 'actor_login') \
            .withColumn("repo_owner", F.split("repo_name", '\/')[0]) \
            .withColumn("project_name", F.split("repo_name", '\/')[1]) \
            .drop("repo_name"), 'id', 'left')

## Total number of push events by user, by project
# and when their first/last push was for that project
agg_push_df = repo_push_df \
    .groupBy('actor_login', 'repo_id', 'repo_owner', 'project_name') \
    .agg(F.sum("distinct_size").alias('number_of_push_events'),\
            F.max('created_at').alias('date_of_last_push'),
            F.min('created_at').alias('date_of_first_push'))

agg_push_df.display()

actor_login,repo_id,repo_owner,project_name,number_of_push_events,date_of_last_push,date_of_first_push
0--key,23730551,0--key,lib,87,01/31/2017 13:00:00,01/11/2017 08:00:00
0-00-4,75778996,0-00-4,0-00-4.github.io,1,01/14/2017 21:00:00,01/14/2017 21:00:00
0-a,75817286,0-a,tensorflow-introduction,6,01/25/2017 02:00:00,01/25/2017 01:00:00
0-wiz-0,25927524,0-wiz-0,mame,1921,01/04/2017 23:00:00,01/04/2017 23:00:00
0-wiz-0,42824785,NetBSD,pkgsrc-wip,223,01/31/2017 22:00:00,01/01/2017 00:00:00
0-wiz-0,73020353,0-wiz-0,flex,44,01/23/2017 15:00:00,01/22/2017 17:00:00
00-00-00,77940436,00-00-00,Roads,1,01/17/2017 16:00:00,01/17/2017 16:00:00
0000101010b,71129853,0000101010b,IndividualSoftwareEngineeringProject,1,01/14/2017 19:00:00,01/14/2017 19:00:00
0000101010b,79169705,0000101010b,Silhouette-Non-photorealistic-rendering,2,01/18/2017 11:00:00,01/16/2017 23:00:00
0000marcell,29162916,0000marcell,Pomodoro-NW,38,01/31/2017 21:00:00,01/02/2017 20:00:00


In [ ]:
# save user activity push data
agg_push_df.write.mode('overwrite') \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/user_push_activity')

In [ ]:
## You can use the aggregation to filter by project name to see what are the most active users by Push Event
# and what did their most recent push event did to the project based on the commit messages 
# for projects that are about any topic, such as "pokemon"
agg_push_df.filter(F.col("project_name").contains("pokemon")).display()

actor_login,repo_id,repo_owner,project_name,number_of_push_events,date_of_last_push,date_of_first_push
2048online,48201076,2048online,pokemon,1,01/29/2017 19:00:00,01/29/2017 19:00:00
AdityaSantoso,79226004,AdityaSantoso,pokemon-smrng,3,01/29/2017 16:00:00,01/21/2017 14:00:00
AleksandrRogachev94,79215341,AleksandrRogachev94,pokemon-scraper-v-000,1,01/18/2017 09:00:00,01/18/2017 09:00:00
Alucky4423,79173015,Alucky4423,pokemon_cmd,1,01/17/2017 00:00:00,01/17/2017 00:00:00
Alx-l,70109079,Alx-l,pokemonyo,19,01/16/2017 00:00:00,01/02/2017 09:00:00
AnthonyAmadeo,77928626,AnthonyAmadeo,pokemon-file-data,1,01/03/2017 15:00:00,01/03/2017 15:00:00
ArchKevin22,76689975,ArchKevin22,pokemon,13,01/22/2017 09:00:00,01/06/2017 05:00:00
Armienn,77997175,Armienn,pokemon-trading-spreadsheet,6,01/04/2017 23:00:00,01/04/2017 09:00:00
Armienn,78103412,Armienn,pokemon-trading-spreadsheet,2,01/05/2017 22:00:00,01/05/2017 10:00:00
Armienn,78189043,Armienn,pokemon-trading-spreadsheet,1,01/06/2017 08:00:00,01/06/2017 08:00:00


## Add language to each event based on commit messages

In [ ]:
### code and language dataframe for the nlp model
codes = ['ar', 'be', 'bg', 'cs', 'da', 'de', 'el', 'en', 'eo', 'es', 'et', 'fa', 'fi', 'fr', 
    'he', 'hi', 'hu', 'ia', 'id', 'is', 'it', 'ja', 'ko', 'la', 'lt', 'lv', 'mk', 'mr', 'nl', 
    'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sr', 'sv', 'tl', 'tr', 'tt', 'uk', 'vi', 'zh', 'Unknown']
langs = ['English', 'English', 'English', 'English', 'English', 'English', 'English', 'English', 
    'English', 'Spanish', 'English', 'English', 'English', 'English', 'English', 'English', 'English', 
    'English', 'English', 'English', 'English', 'Japanese', 'Korean', 'English', 'English', 'English', 
    'English', 'English', 'English', 'Polish', 'Portuguese', 'English', 'Russian', 'English', 'English', 
    'English', 'English', 'English', 'English', 'English', 'English', 'Vietnamese', 'Chinese', 'Unknown']

annuaire = spark.createDataFrame(zip(codes, langs), ['code','language'])
annuaire.display()

code,language
ar,English
be,English
bg,English
cs,English
da,English
de,English
el,English
en,English
eo,English
es,Spanish


In [ ]:
### 
main = main_df.select("created_at", "id", "type", "actor_login", "org_login")

# Select the event id and the comment for all Dataframes event types that have comments 
commits = commit_comment_event_df.select("id", "comment_body") \
    .withColumnRenamed("comment_body", "comment")
issues = issue_comment_event_df.select("id", "comment_body") \
    .withColumnRenamed("comment_body", "comment")
push = push_event_df_commits_df.groupby("fact_id") \
    .agg(F.concat_ws(',',F.collect_list("message")) \
    .alias("comment")).withColumnRenamed("fact_id","id").select("id","comment")
pull = pull_request_review_comment_event_df \
    .select("id","comment_body") \
    .withColumnRenamed("comment_body", "comment")

# Union all the id/comment dataframes for all the events with comments
comment_data = commits.union(issues).union(push).union(pull)


filt_main = main \
    .where((main.type == "PushEvent") |
        (main.type == "IssueCommentEvent") |
        (main.type == "CommitCommentEvent") |
        (main.type == "PullRequestReviewCommentEvent"))
all_comments = filt_main.join(comment_data,"id")
all_comments.persist(StorageLevel.DISK_ONLY)

Out[10]: DataFrame[id: string, created_at: string, type: string, actor_login: string, org_login: string, comment: string]

In [ ]:
# initializing document assembler 
documentAssembler = DocumentAssembler()\
.setInputCol("comment")\
.setOutputCol("document")

#initializing language detector 
language_detector = LanguageDetectorDL.pretrained("ld_wiki_tatoeba_cnn_43", "xx")\
.setInputCols(["document"])\
.setOutputCol("lang")\
.setThreshold(0.1)\
.setCoalesceSentences(True)

languagePipeline = Pipeline(stages=[
 documentAssembler, 
 language_detector
])

results = languagePipeline.fit(all_comments).transform(all_comments)


ready_results = results.select("id",results.lang.result[0]).withColumnRenamed("lang.result[0]","code")

event_languages = all_comments.join(ready_results,"id").join(annuaire,"code", "left")


event_languages.display()

ld_wiki_tatoeba_cnn_43 download started this may take some time.
Approximate size to download 7.5 MB
[ | ][OK!]

code,id,created_at,type,actor_login,org_login,comment,language
es,5088648630,2017-01-01T00:00:09Z,PushEvent,EdgarVaguencia,null,Actualizando el clima :partly_sunny:,Spanish
la,5088649215,2017-01-01T00:00:30Z,PushEvent,shgtkshruch,null,Remove imac image,English
en,5088649225,2017-01-01T00:00:30Z,PushEvent,mrocklin,null,trivial change to trigger circle-ci,English
en,5088650021,2017-01-01T00:01:27Z,PushEvent,SijiaDavis,null,add run,English
en,5088650342,2017-01-01T00:01:54Z,PushEvent,AlexGyver,null,Add files via upload,English
en,5088651123,2017-01-01T00:02:49Z,IssueCommentEvent,titusfortner,null,"Released this with 0.3.0, but still interested in thoughts",English
en,5088651400,2017-01-01T00:03:11Z,PushEvent,rmessing,null,added config.logger to production.rb,English
es,5088651851,2017-01-01T00:03:41Z,PushEvent,pixelzoom,phetsims,TODO,Spanish
en,5088651976,2017-01-01T00:03:51Z,PushEvent,tylerfilla,null,Update footer.html,English
fr,5088652098,2017-01-01T00:04:01Z,PushEvent,kiang,null,update datasets @ 2017-01-01 08:00:00,English


In [ ]:
event_languages.persist(StorageLevel.DISK_ONLY)

Out[12]: DataFrame[code: string, id: string, created_at: string, type: string, actor_login: string, org_login: string, comment: string, language: string]

In [ ]:
language_stats = event_languages.groupBy('type') \
    .pivot('language') \
    .count() \
    .fillna(0) \
    .orderBy('type') \
    .withColumnRenamed('actor_display_login', 'user')

language_stats.persist(StorageLevel.DISK_ONLY)

display(language_stats.drop("null"))

type,Chinese,English,Japanese,Korean,Polish,Portuguese,Russian,Spanish,Unknown,Vietnamese
CommitCommentEvent,2489,64216,882,1298,552,660,2186,664,773,51
IssueCommentEvent,50159,2211923,9777,14098,5047,10462,13985,6932,9672,473
PullRequestReviewCommentEvent,6861,521135,5509,4088,1105,6245,16671,1598,1605,6091
PushEvent,361787,13221139,48874,42414,87082,242091,53303,170037,112134,5261


In [ ]:
display(language_stats.drop("null"))

type,Chinese,English,Japanese,Korean,Polish,Portuguese,Russian,Spanish,Unknown,Vietnamese
CommitCommentEvent,2489,64216,882,1298,552,660,2186,664,773,51
IssueCommentEvent,50159,2211923,9777,14098,5047,10462,13985,6932,9672,473
PullRequestReviewCommentEvent,6861,521135,5509,4088,1105,6245,16671,1598,1605,6091
PushEvent,361787,13221139,48874,42414,87082,242091,53303,170037,112134,5261


In [ ]:
# writing to a file in gold 
event_languages.write.mode("overwrite") \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/event_languages_df')

In [ ]:
# writing to a file in gold 
language_stats.write.mode("overwrite") \
    .parquet('abfs://gold-layer@ade20220919.dfs.core.windows.net/miniro/language_stats_df')